In [ ]:
TEST = False
DOWNLOAD = True
COLAB = False

## Elastic Search Server

In [ ]:
if DOWNLOAD: 
    !pip install elasticsearch

     |████████████████████████████████| 385 kB 5.3 MB/s 


In [ ]:
import os
import time
from elasticsearch import Elasticsearch

In [ ]:
if DOWNLOAD:
    !wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.6.2-linux-x86_64.tar.gz -q
    !tar -xzf elasticsearch-7.6.2-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.6.2

In [ ]:
%%bash --bg

sudo -H -u daemon elasticsearch-7.6.2/bin/elasticsearch

Starting job # 0 in a separate thread.


In [ ]:
# Sleep for few seconds to let the instance start.
time.sleep(20)

In [ ]:
%%bash

ps -ef | grep elasticsearch

root         118     116  0 22:32 ?        00:00:00 sudo -H -u daemon elasticsearch-7.6.2/bin/elasticsearch
daemon       119     118 99 22:32 ?        00:00:30 /content/elasticsearch-7.6.2/jdk/bin/java -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=COMPAT -Xms1g -Xmx1g -XX:+UseConcMarkSweepGC -XX:CMSInitiatingOccupancyFraction=75 -XX:+UseCMSInitiatingOccupancyOnly -Djava.io.tmpdir=/tmp/elasticsearch-16190466211988066045 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:filecount=32,filesize=64m -XX:MaxDirectMe

## Global Variables

In [ ]:
if COLAB:
    from google.colab import drive
    drive.mount('/content/drive/')

    %cd /content/drive/MyDrive/Colab Notebooks/NLP_Project/src

Mounted at /content/drive/
/content/drive/MyDrive/Colab Notebooks/NLP_Project/src


In [ ]:
import numpy as np
import nltk

data_path = "../data/"
articles_path = data_path+"articles/articles/"
test_data_path = data_path+"QAData.txt"
test_input_path = data_path+"Sample_Questions_and_AnswerDocs/sample_questions.txt"
test_output_path = data_path+"Sample_Questions_and_AnswerDocs/sample_questions.xlsx"
java_path = "D:/JDK/jdk-13.0.1/bin/java.exe"

if TEST:
    with open(articles_path+"400.txt", 'r') as file:
        test_articles = file.read()
        file.close()
    test_question = 'What was the capital of the Safavid Dynasty?'

## TASK 1

In [ ]:
if DOWNLOAD:
    #environment setup
    nltk.download("punkt")
    nltk.download("averaged_perceptron_tagger")
    nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
import os
import nltk
from nltk.corpus import wordnet as wn
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from collections import  defaultdict
from nltk.parse.stanford import StanfordDependencyParser
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

os.environ['JAVAHOME'] = java_path

tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

def articleDir(filepath):        
    pathDir = os.listdir(filepath)
    return pathDir

def createSentenceList(data):
    SentenceList = []
    SentenceList = sent_tokenize(data)
    return SentenceList

def createTokenList(data):
    TokenList = []
    TokenList = word_tokenize(data)
    return TokenList

def Tokenize(filepath):
    pathDir = articleDir(filePath) 
    articleSentenceList = []
    articleTokenList = []
    articleSentenceList2 = []
    articleTokenList2 = []
    for article in pathDir:
        if article == '.DS_Store':
            continue
        txtname = filepath+article
        file = open(txtname,encoding='utf-8')
        data = file.read()
        sentenceList = createSentenceList(data)
        articleSentenceList.append(sentenceList)
        articleSentenceList2.extend(sentenceList)
        tokenList = createTokenList(data)
        articleTokenList.append(tokenList)
        articleTokenList2.extend(tokenList)
    return articleSentenceList, articleSentenceList2, articleTokenList, articleTokenList2
    

def Lemmatize(tokenlist):
    lemmalist = []
    lemmatizer = WordNetLemmatizer()
    for token, tag in pos_tag(tokenlist):
        lemma = lemmatizer.lemmatize(token, tag_map[tag[0]])
        lemmalist.append(lemma)    
    return lemmalist

def dependencyParse(sentencelist):
    jar_path = 'nltk/stanford-corenlp-4.2.2/stanford-corenlp-4.2.2.jar'
    models_jar_path = 'nltk/stanford-corenlp-4.2.2-models-english.jar'
    parser = StanfordDependencyParser(path_to_jar = jar_path, path_to_models_jar = models_jar_path)

    for sentence in sentencelist:
        result = parser.raw_parse(sentence)
        dependency = result.__next__()
        print ("{:<15} | {:<10} | {:<10} | {:<15} | {:<10}".format('Head', 'Head POS','Relation','Dependent', 'Dependent POS'))
        print ("-" * 75)
        for dep in list(dependency.triples()):
            print ("{:<15} | {:<10} | {:<10} | {:<15} | {:<10}".format(str(dep[0][0]),str(dep[0][1]), str(dep[1]), str(dep[2][0]),str(dep[2][1])))
        print('\n')

In [ ]:
#part of speech tag 
from nltk import tokenize
from nltk import pos_tag

def tag_document(doc:str) -> list:
    sentences = tokenize.sent_tokenize(doc)
    tagged_sentences = list()
    for sentence in sentences: 
        tagged_sentences.append(tag_sentence(sentence))
    return tagged_sentences
def tag_sentence(sentence:str) -> list:
    words = tokenize.word_tokenize(sentence)
    return pos_tag(words)
    

In [ ]:
poses = [wn.NOUN, wn.VERB, wn.ADJ, wn.ADV]

In [ ]:
def wordnetSynonyms(tokenlist):
    synonyms = []
    antonyms = []
    for i in range(0, len(tokenlist)):
        wn_word = tokenlist[i].replace(' ', '_')
        for pos in poses:
            for syn_set in wn.synsets(wn_word, pos):
                for l in syn_set.lemmas():
                    synonyms.append([])
                    synonyms[i].append(l.name())
                    #if l.antonyms(): 
                        #antonyms.append([])
                        #antonyms[i].append(l.antonyms()[0].name())   
        if set(synonyms[i])!=set():          
            print(set(synonyms[i])) 
    return synonyms

In [ ]:
porter = PorterStemmer()
def stem(word):
    stem=porter.stem(word)
    #print(stem)
    return stem

def Lemma(word):
    lemmaword = []
    lemmatizer = WordNetLemmatizer()
    for token, tag in pos_tag(word):
        lemma = lemmatizer.lemmatize(token, tag_map[tag[0]])
        lemmaword.append(lemma)    
    #print(lemmaword)
    return lemmaword

def Synonyms(word):
    synonyms = []
    wn_word = word.replace(' ', '_')
    for pos in poses:
        for syn_set in wn.synsets(wn_word, pos):
            for l in syn_set.lemmas():
                synonyms.append(l.name())
    #print(set(synonyms))
    return list(set(synonyms)) 

def Antonyms(word):
    antonyms=[]
    wn_word = word.replace(' ', '_')
    for pos in poses:
        for syn_set in wn.synsets(wn_word, pos):
            for l in syn_set.lemmas():
                if l.antonyms():
                    antonyms.append(l.antonyms()[0].name())
    return list(set(antonyms))  

def Hyponyms(word):
    hyponyms = []
    wn_word = word.replace(' ', '_')
    for pos in poses:
        for syn_set in wn.synsets(wn_word, pos):
            for l in syn_set.hyponyms():
                hyponyms.append(l.name().split('.')[0])
    #print(set(hyponyms))
    return list(set(hyponyms))

def Hypernyms(word):
    hypernyms = []
    wn_word = word.replace(' ', '_')
    for pos in poses:
        for syn_set in wn.synsets(wn_word, pos):
            for l in syn_set.hypernyms():
                hypernyms.append(l.name().split('.')[0])
    #print(set(hypernyms))
    return list(set(hypernyms))
    
def Meronyms(word):
    meronyms = []
    wn_word = word.replace(' ', '_')
    for pos in poses:
        for syn_set in wn.synsets(wn_word, pos):
            for l in syn_set.part_meronyms():
                meronyms.append(l.name().split('.')[0])
    #print(set(meronyms))
    return list(set(meronyms))

def Holonyms(word):
    holonyms = []
    wn_word = word.replace(' ', '_')
    for pos in poses:
        for syn_set in wn.synsets(wn_word, pos):
            for l in syn_set.part_holonyms():
                holonyms.append(l.name().split('.')[0])
    #print(set(holonyms))
    return list(set(holonyms))

In [ ]:
def postag1(questionsentence):
    word=word_tokenize(questionsentence)
    return pos_tag(word)


In [ ]:
# if __name__ == '__main__':
#     filePath="articles"
#     articleSentenceList, articleSentenceList2, articleTokenList, articleTokenList2 = Tokenize(filePath)
#     LemmaList = Lemmatize(articleTokenList2)
#     #print(LemmaList)
#     #dependencyParse(articleSentenceList2)
#     #synonym=wordnetSynonyms(articleTokenList2)

## Run Task 1

In [ ]:
def outputSentenceAndWord(articleSentenceList,articleTokenList,LemmaList):
    file = open('sentence.txt', 'w', encoding='utf-8')
    file.write('sentence:'+ '\n')
    for sentence in articleSentenceList:
        file.write(sentence+ '\n')
    file.close()
    
    file = open('word.txt', 'w', encoding='utf-8')
    file.write('word:'+ '\n')
    for word in articleTokenList:
        file.write(word+ '\n')
    file.close()
    
    file = open('lemma.txt', 'w', encoding='utf-8')
    file.write('lemma:'+ '\n')
    for lemma in LemmaList:
        file.write(lemma+ '\n')
    file.close()
    
    file = open('POStag.txt', 'w', encoding='utf-8')
    file.write('POStag:'+ '\n')
    for sentence in articleSentenceList:
        taglist=postag1(sentence)
        for each in taglist:
            file.write(each[0]+'_'+each[1]+ '   ')
        file.write('\n')
    file.close()
    
    file = open('Hypernyms.txt', 'w', encoding='utf-8')
    file.write('Hypernyms:'+ '\n')
    for word in articleTokenList:
        file.write(word+ ':  ') 
        hypernyms=Hypernyms(word)
        for each in hypernyms:
            file.write(each+ '  ') 
        file.write('\n')
    file.close()
    
    file = open('Hyponyms.txt', 'w', encoding='utf-8')
    file.write('Hyponyms:'+ '\n')
    for word in articleTokenList:
        file.write(word+ ':  ') 
        hyponyms=Hyponyms(word)
        for each in hyponyms:
            file.write(each+ '  ') 
        file.write('\n')
    file.close()
    
    file = open('Meronyms.txt', 'w', encoding='utf-8')
    file.write('Meronyms:'+ '\n')
    for word in articleTokenList:
        file.write(word+ ':  ') 
        meronyms=Meronyms(word)
        for each in meronyms:
            file.write(each+ '  ') 
        file.write('\n')
    file.close()
    
    file = open('Holonyms.txt', 'w', encoding='utf-8')
    file.write('Holonyms:'+ '\n')
    for word in articleTokenList:
        file.write(word+ ':  ') 
        holonyms=Holonyms(word)
        for each in holonyms:
            file.write(each+ '  ') 
        file.write('\n')
    file.close()

def outputDependencyParse(sentencelist):
    jar_path = 'nltk/stanford-corenlp-4.2.2/stanford-corenlp-4.2.2.jar'
    models_jar_path = 'nltk/stanford-corenlp-4.2.2-models-english.jar'
    parser = StanfordDependencyParser(path_to_jar = jar_path, path_to_models_jar = models_jar_path)
    file = open('DependencyParse.txt', 'w', encoding='utf-8')
    for sentence in sentencelist:
        result = parser.raw_parse(sentence)
        dependency = result.__next__()
        file.write("{:<15} | {:<10} | {:<10} | {:<15} | {:<10}".format('Head', 'Head POS','Relation','Dependent', 'Dependent POS'))       
        file.write('\n'+"-" * 75+'\n')
        for dep in list(dependency.triples()):
            file.write("{:<15} | {:<10} | {:<10} | {:<15} | {:<10}".format(str(dep[0][0]),str(dep[0][1]), str(dep[1]), str(dep[2][0]),str(dep[2][1])))
            file.write('\n')
        file.write('\n')

if TEST:
    filePath="articles"
    articleSentenceList, articleSentenceList2, articleTokenList, articleTokenList2 = Tokenize(filePath)
    LemmaList = Lemmatize(articleTokenList2)
    outputSentenceAndWord(articleSentenceList2,articleTokenList2,LemmaList)
    #outputDependencyParse(articleSentenceList2)

## TASK 2

### Client

In [ ]:
from elasticsearch import Elasticsearch

config = {'host':'localhost', 'port':9200}
es = Elasticsearch([config])

# test connection
es.ping()

True

#### parse_record_to_doc

In [ ]:
from collections import  defaultdict
from nltk import tokenize

In [ ]:
def parse_record_to_doc(articles_path):
    '''
    parse into documents

    Args: 
        articles_path (str) - path of the wiki articles
    
    Return: 
        wiki_articles (list) - list of dicts, where each dict contains two keys: 'document_id' and 'document_text'.
    '''
    wiki_articles = defaultdict()
    for item in os.listdir(articles_path):
        if item == '.DS_Store':
            continue
        with open(articles_path + item, 'r') as file:
            article_id = item.split('.')[0]
            wiki_articles[article_id] = file.read()
            file.close()
    wiki_articles = [{'document_id':id, 'document_text':text}\
                     for id, text in wiki_articles.items()]
    return wiki_articles

#### parse_record_to_n-gram

In [ ]:
from nltk.tokenize import sent_tokenize

In [ ]:
def createSentenceList(data):
    SentenceList = []
    SentenceList = sent_tokenize(data)
    return SentenceList

In [ ]:
def parse_record_to_n_gram(articles, n):
    '''
    parse into documents with sentence bigram
    Args: 
        articles_path (str) - path of the wiki articles
    Return: 
        wiki_articles (list) - list of dicts, where each dict contains three keys: 'document_id', 'document_subid' and 'document_text'.
    '''
    wiki_articles = defaultdict()
    
    article_id_list = []
    article_subid_list = []
    article_n_gram_list = []
    sentence_n_grams=[]
    for article in articles:
        temp_dict = dict()
        article_id = article['document_id']
        text=article['document_text']
        sentenceList = createSentenceList(text)
        #creates the n-gram
        sentence_n_grams = zip(*[sentenceList[i:] for i in range(n)])
        sentence_n_grams = [' '.join(i) for i in sentence_n_grams]
        
        article_id_list.extend([article_id]*len(sentence_n_grams))
        article_subid_list.extend([i for i in range(len(sentence_n_grams))])
        article_n_gram_list.extend(sentence_n_grams)
    wiki_articles = zip(article_id_list, article_subid_list, article_n_gram_list)
    wiki_articles = [{'document_id':id, 'document_subid':subid, 'document_text':text}\
                     for id, subid, text in wiki_articles]
    return wiki_articles

In [ ]:
articles_by_doc = parse_record_to_doc(articles_path)
articles_by_n_grams = parse_record_to_n_gram(articles_by_doc, 3)

#### populate_index

In [ ]:
from tqdm.notebook import tqdm

def populate_index(es_obj, index_name, evidence_corpus):
    '''
    Loads records into an existing Elasticsearch index

    Args:
        es_obj (elasticsearch.client.Elasticsearch) - Elasticsearch client object
        index_name (str) - Name of index
        evidence_corpus (list) - List of dicts containing data records
    '''
    for i, rec in enumerate(tqdm(evidence_corpus)):
        #index_status = es_obj.index(index=index_name, id=i, body=rec)
        try:
            index_status = es_obj.index(index=index_name, id=i, body=rec)
        except:
            print(f'Unable to load document {i}.')
            
    n_records = es_obj.count(index=index_name)['count']
    print(f'Succesfully loaded {n_records} into {index_name}')
    
    return

In [ ]:
!curl -XDELETE localhost:9200/_all

{"acknowledged":true}

#### Populate 'doc_index'

In [ ]:
# index_config = {
#     "settings": {
#         "analysis": {
#             "analyzer": {
#                 "stop_stem_analyzer": {
#                     "type": "custom",
#                     "tokenizer": "standard",
#                     "filter":[
#                         "lowercase",
#                         "stop",
#                         "snowball"
#                     ]
                    
#                 }
#             }
#         }
#     },
#     "mappings": {
#         "dynamic": "strict", 
#         "properties": {
#             "document_id": {"type": "text", "analyzer": "standard_analyzer"},
#             "document_text": {"type": "text", "analyzer": "standard_analyzer"}
#             }
#         }
#     }
index_config ={
  "settings": {
     "index" : {
        "analysis" : {
            "analyzer" : {
                "synonym" : {
                    "tokenizer" : "whitespace",
                    "filter" : ["synonym"]
                }
            },
            "filter" : {
                "synonym" : {
                    "type" : "synonym",
                    "format" : "wordnet",
                    "synonyms_path" : "analysis/wordnet_synonyms.txt"
                }
            }
        }
     }
  },
  "mappings": {
     "typename": {
        "fieldname": {
           "type": "string",
           "analyzer": "synonym"
        }
     },
     "dynamic": "strict", 
     "properties": {
         "document_id": {"type": "text", "analyzer": "standard_analyzer"},
         "document_text": {"type": "text", "analyzer": "standard_analyzer"}
         }
    }
}

doc_index = 'doc_index'
es.indices.create(index=doc_index, body=index_config, ignore=400)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information


{'error': {'caused_by': {'reason': '/content/elasticsearch-7.6.2/config/analysis/wordnet_synonyms.txt',
   'type': 'no_such_file_exception'},
  'reason': 'IOException while reading synonyms_path_path: /content/elasticsearch-7.6.2/config/analysis/wordnet_synonyms.txt',
  'root_cause': [{'reason': 'IOException while reading synonyms_path_path: /content/elasticsearch-7.6.2/config/analysis/wordnet_synonyms.txt',
    'type': 'illegal_argument_exception'}],
  'type': 'illegal_argument_exception'},
 'status': 400}

In [ ]:
populate_index(es_obj=es, index_name=doc_index, evidence_corpus=articles_by_doc)

  0%|          | 0/30 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  from ipykernel import kernelapp as app


Succesfully loaded 0 into doc_index


#### Populate 'n_gram_index'

In [ ]:
# index_config = {
#     "settings": {
#         "analysis": {
#             "analyzer": {
#                 "stop_stem_analyzer": {
#                     "type": "custom",
#                     "tokenizer": "standard",
#                     "filter":[
#                         "lowercase",
#                         "stop",
#                         "snowball"
#                     ]
                    
#                 }
#             }
#         }
#     },
#     "mappings": {
#         "dynamic": "strict", 
#         "properties": {
#             "document_id": {"type": "text", "analyzer": "standard_analyzer"},
#             "document_subid": {"type": "text", "analyzer": "standard_analyzer"},
#             "document_text": {"type": "text", "analyzer": "standard_analyzer"}
#             }
#         }
#     }
index_config ={
  "settings": {
     "index" : {
        "analysis" : {
            "analyzer" : {
                "synonym" : {
                    "tokenizer" : "whitespace",
                    "filter" : ["synonym"]
                }
            },
            "filter" : {
                "synonym" : {
                    "type" : "synonym",
                    "format" : "wordnet",
                    "synonyms_path" : "analysis/wordnet_synonyms.txt"
                }
            }
        }
     }
  },
  "mappings": {
     "typename": {
        "fieldname": {
           "type": "string",
           "analyzer": "synonym"
        }
     },
     "dynamic": "strict", 
     "properties": {
         "document_id": {"type": "text", "analyzer": "standard_analyzer"},
         "document_subid": {"type": "text", "analyzer": "standard_analyzer"},
         "document_text": {"type": "text", "analyzer": "standard_analyzer"}
         }
    }
}

n_gram_index = 'n_gram_index'
es.indices.create(index=n_gram_index, body=index_config, ignore=400)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:64: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information


{'error': {'caused_by': {'reason': '/content/elasticsearch-7.6.2/config/analysis/wordnet_synonyms.txt',
   'type': 'no_such_file_exception'},
  'reason': 'IOException while reading synonyms_path_path: /content/elasticsearch-7.6.2/config/analysis/wordnet_synonyms.txt',
  'root_cause': [{'reason': 'IOException while reading synonyms_path_path: /content/elasticsearch-7.6.2/config/analysis/wordnet_synonyms.txt',
    'type': 'illegal_argument_exception'}],
  'type': 'illegal_argument_exception'},
 'status': 400}

In [ ]:
populate_index(es_obj=es, index_name=n_gram_index, evidence_corpus=articles_by_n_grams)

  0%|          | 0/7060 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  from ipykernel import kernelapp as app


Succesfully loaded 7060 into n_gram_index


### Function for Evaluation 

#### Format test data

In [ ]:
import json
import re

In [ ]:
def load_test_data(test_data_path):
    '''
    Load test data. It will print out any ques-ans pair that is discarded in the process. 

    Args:
        test_data_path (str) - location of the test data
    
    Returns:
        test_data_res (dict) - Each entry of the dict corresponding to a list of ques-ans tuples.  
    '''
    with open(test_data_path, 'r') as file:
        test_data = file.read()
        file.close()

    test_data = test_data.split('\n')
    test_data_res = dict()
    for line in test_data:
        num = re.search(r'[0-9]{1,4}', line).group(0)
        if num in test_data_res:
            continue

        test_data_res[num] = list()
        questions = re.findall(r'\(.*?\),|\(.*?\)\]', line)
        for q in questions:
            q1 = q
            q = re.findall(r"\'.*?\'|\".*?\"", q1)
            if len(q) != 2:
                print(q1)
                print(q)
                continue
            q[0] = q[0][1:-1]
            q[1] = q[1][1:-1]
            q = tuple(q)
            test_data_res[num].append(q)
        #print(len(test_data_res[num]))
    return test_data_res

In [ ]:
test_data = load_test_data(test_data_path)

('What two formats replaced the 78?', '33 1â„3 rpm (often just referred to as the 33 rpm),
["'What two formats replaced the 78?'"]
('What races are considered for full citizenship in Burma?', 'national races such as Kachin, Kayah (Karenni),
["'What races are considered for full citizenship in Burma?'"]
('What is episodic memory?', '"which attempts to capture information such as \'what\', \'when\' and \'where\''),
["'What is episodic memory?'", '\'"which attempts to capture information such as \\\'', "', \\'", "' and \\'", "''"]
('What is the treatment of choice for diarrhea?', 'Oral rehydration solution (ORS),
["'What is the treatment of choice for diarrhea?'"]


#### Function for Evaluation

In [ ]:
def evalution_ans(y_true, y_predict):
    if y_true in y_predict:
        return 1
    else:
        return 0

In [ ]:
def evaluate_filter(filter_fn, ques_ans_pair, document_number, display_miss=False, display_correct=False):
    '''
    evaluate the effectiveness of a filter on a single ques_ans_pair

    Args:
        ques_ans_pair (tuple) - a ques-ans tuple
        document_number (int) - where the answer should located
        n (int) - number of answers the filter should return 
    Returns:
        hit (int) - 1 if the filter's return contains the correct ans; 0 otherwise

    '''
    res = filter_fn(ques_ans_pair[0])

    for ans in res:
        if document_number == ans[0] and evalution_ans(ques_ans_pair[1], ans[1]):
            hit = 1
            if display_correct:
                print_correct(document_number, ques_ans_pair, res)
            return hit
        else:
            pass
    if display_miss:
        print_miss(document_number, ques_ans_pair, res)

    hit = 0
    return hit

In [ ]:
def evaluate_all(filter_fn, test_data, display_miss=False, display_correct=False):
    correct_count = 0
    total_count = 0
    for doc_num in test_data:
        for ques_ans_pair in test_data[doc_num]:
            total_count += 1
            correct_count += evaluate_filter(filter_fn, ques_ans_pair, doc_num, display_miss=display_miss, display_correct=display_correct)
    return correct_count, total_count

### Pipline

#### Extract doc

In [ ]:
def search_doc(es_obj, index_name, question_text, n_results):
    '''
    Execute an Elasticsearch query on a specified index
    
    Args:
        es_obj (elasticsearch.client.Elasticsearch) - Elasticsearch client object
        index_name (str) - Name of index to query
        query (dict) - Query DSL
        n_results (int) - Number of results to return
        
    Returns
        res - Elasticsearch response object
    '''
    
    # construct query
    query = {
            'match': {
                    'document_text': question_text
                    }
            }
    
    res = es_obj.search(index=index_name, query=query, size=n_results)
    
    return res

In [ ]:
def extract_doc(question, n):
    '''
    Extract top n doc that have the highest similarity with the question

    Args:
        question (str) - a question string
        n (int) - number of documents the function should return

    Retures:
        res (list) - a list of ques-ans tuples  
    '''
    #proper_noun = extract_proper_noun(question)
    #question = question + ' ' + (proper_noun + ' ') * 10
    res = search_doc(es_obj=es, index_name=doc_index, question_text=question, n_results=n)
    res = [(hit['_source']['document_id'], hit['_source']['document_text']) for hit in res['hits']['hits']]
    return res

#### Extract n-grams

In [ ]:
def search_n_grams(es_obj, index_name, doc_range, question_text, n_results):
    '''
    Execute an Elasticsearch query on a specified index
    
    Args:
        es_obj (elasticsearch.client.Elasticsearch) - Elasticsearch client object
        index_name (str) - Name of index to query
        query (dict) - Query DSL
        n_results (int) - Number of results to return
        
    Returns
        res - Elasticsearch response object
    '''
    
    # construct query
    query = {
            'bool':{
                    'must':[
                        {'match': {'document_text': question_text}}    
                        ],
                    'filter':[
                        {'terms':{'document_id':doc_range}}
                        ]
                    }
            }
    
    res = es_obj.search(index=index_name, query=query, size=n_results)
    
    return res

In [ ]:
from nltk import tokenize
from nltk import pos_tag
def extract_proper_noun(sentence):
    '''
    Extract proper noun from a sentence

    Args:
        sentence (str) - a sentence

    Returns:
        res (str) - Subset of the sentence containing only proper noun
    '''
    words = tokenize.word_tokenize(sentence)
    words = pos_tag(words)
    proper_noun = list()
    for w in words:
        if w[1] == 'NNP':
            proper_noun.append(w[0])
    
    res = ' '.join(proper_noun)
    return res

In [ ]:
def extract_n_grams(doc_range, question, n):
    '''
    Extract top n doc that have the highest similarity with the question

    Args:
        question (str) - a question string
        n (int) - number of documents the function should return

    Retures:
        res (list) - a list of ques-ans tuples  
    '''
    #proper_noun = extract_proper_noun(question)
    #question = question + ' ' + (proper_noun + ' ') * 10
    res = search_n_grams(es_obj=es, index_name=n_gram_index, doc_range=doc_range, question_text=question, n_results=n)
    res = [(hit['_source']['document_id'], hit['_source']['document_text']) for hit in res['hits']['hits']]
    return res

#### Choose Doc

In [ ]:
from collections import Counter
def choose_doc(id_text_pairs):
    doc_id = [i[0] for i in id_text_pairs]
    c = Counter(doc_id)
    most_common_doc = c.most_common(1)[0][0]

    new_list = []
    for id_text in id_text_pairs:
        if id_text[0] == most_common_doc:
            new_list.append(id_text)
    return new_list

#### Filter Proper Noun

In [ ]:
def postag(questionsentence):
    word=word_tokenize(questionsentence)
    return pos_tag(word)

def processTag(taglist):
    keywordNNP=[]
    for i in range(len(taglist)):
        if taglist[i][1]=='NNP' or taglist[i][1]=='NNPS':
            keywordNNP.append(taglist[i][0])
    return keywordNNP

def extractNNP(questionsentence):
    tag_tuple = postag(questionsentence)
    keywordNNP = set(processTag(tag_tuple))

    return keywordNNP

In [ ]:
def filter_proper_noun(question_sentence, id_text_pairs):
    tag_tuple = postag(question_sentence)
    keywordNNP = set(processTag(tag_tuple))

    new_list = []
    for id_text_pair in id_text_pairs:
        temp = word_tokenize(id_text_pair[1])
        temp = set(temp)
        if keywordNNP.issubset(temp):
            new_list.append(id_text_pair)
    return new_list

In [ ]:
if TEST:
    question_sentence = 'What was the capital of the Safavid Dynasty?'
    id_text_pairs = [('400', 'Establishment of the Safavid Dynasty in 1501, converted the Iranian people from Sunni Islam to Twelver Shia Islam, and made Twelver Shia Islam the official religion of Iran. Safavid conversion of Iran from Sunnism to Shiism marked one of the most important turning points in Iranian and Muslim history. Starting in 1736 under Nader Shah, Iran reached its greatest territorial extent since the Sassanid Empire, briefly possessing what was arguably the most powerful empire at the time.'), ('400', 'In 1387, Timur ordered the complete massacre of Isfahan, reportedly killing 70,000 citizens. The Ilkhans and the Timurids soon came to adopt the ways and customs of the Iranians, choosing to surround themselves with a culture that was distinctively Iranian. By the 1500s, Ismail I from Ardabil, established the Safavid Dynasty, with Tabriz as the capital.'), ('400', 'By the 1500s, Ismail I from Ardabil, established the Safavid Dynasty, with Tabriz as the capital. Beginning with Azerbaijan, he subsequently extended his authority over all of the Iranian territories, and established an intermittent Iranian hegemony over the vast relative regions, reasserting the Iranian identity within large parts of the Greater Iran. Iran was predominantly Sunni, but Ismail instigated a forced conversion to the Shia branch of Islam, by which the Shia Islam spread throughout the Safavid territories in the Caucasus, Iran, Anatolia, and Mesopotamia.'), ('400', 'The Ilkhans and the Timurids soon came to adopt the ways and customs of the Iranians, choosing to surround themselves with a culture that was distinctively Iranian. By the 1500s, Ismail I from Ardabil, established the Safavid Dynasty, with Tabriz as the capital. Beginning with Azerbaijan, he subsequently extended his authority over all of the Iranian territories, and established an intermittent Iranian hegemony over the vast relative regions, reasserting the Iranian identity within large parts of the Greater Iran.'), ('6', 'Meanwhile, the Chinese Ming dynasty fell to the rebellion of Li Zicheng (1606–1645) in 1644, yet his short-lived Shun dynasty was crushed by the Manchu invasion and the Han Chinese general Wu Sangui (1612–1678). China Daily states that when the following Qing dynasty replaced the Ming dynasty, it merely "strengthened administration of Tibet." However, Kolmaš states that the Dalai Lama was very observant of what was going on in China and accepted a Manchu invitation in 1640 to send envoys to their capital at Mukden in 1642, before the Ming collapsed.'), ('6', 'China Daily states that when the following Qing dynasty replaced the Ming dynasty, it merely "strengthened administration of Tibet." However, Kolmaš states that the Dalai Lama was very observant of what was going on in China and accepted a Manchu invitation in 1640 to send envoys to their capital at Mukden in 1642, before the Ming collapsed. Dawa Norbu, William Rockhill, and George N. Patterson write that when the Shunzhi Emperor (r. 1644–1661) of the subsequent Qing dynasty invited the fifth Dalai Lama Lozang Gyatso to Beijing in 1652, Shunzhi treated the Dalai Lama as an independent sovereign of Tibet.'), ('400', 'Iran became a major contributor to the Islamic Golden Age, producing many influential scientists, scholars, artists, and thinkers. Establishment of the Safavid Dynasty in 1501, converted the Iranian people from Sunni Islam to Twelver Shia Islam, and made Twelver Shia Islam the official religion of Iran. Safavid conversion of Iran from Sunnism to Shiism marked one of the most important turning points in Iranian and Muslim history.'), ('304', 'The Shunga Empire or Shunga Empire was an ancient Indian dynasty from Magadha that controlled vast areas of the Indian subcontinent from around 187 to 78 BCE. The dynasty was established by Pushyamitra Shunga, after the fall of the Maurya Empire. Its capital was Pataliputra, but later emperors such as Bhagabhadra also held court at Besnagar, modern Vidisha in Eastern Malwa.'), ('6', 'However, Kolmaš states that the Dalai Lama was very observant of what was going on in China and accepted a Manchu invitation in 1640 to send envoys to their capital at Mukden in 1642, before the Ming collapsed. Dawa Norbu, William Rockhill, and George N. Patterson write that when the Shunzhi Emperor (r. 1644–1661) of the subsequent Qing dynasty invited the fifth Dalai Lama Lozang Gyatso to Beijing in 1652, Shunzhi treated the Dalai Lama as an independent sovereign of Tibet. Patterson writes that this was an effort of Shunzhi to secure an alliance with Tibet that would ultimately lead to the establishment of Manchu rule over Mongolia.'), ('400', 'Safavid conversion of Iran from Sunnism to Shiism marked one of the most important turning points in Iranian and Muslim history. Starting in 1736 under Nader Shah, Iran reached its greatest territorial extent since the Sassanid Empire, briefly possessing what was arguably the most powerful empire at the time. During the 19th century, Iran irrevocably lost swaths of its territories in the Caucasus which made part of the concept of Iran for centuries, to neighboring Imperial Russia.')]

    #question_sentence='What latitudes does Iran lie between?'
    #id_text_pairs=[('400','Establishment of the Safavid Dynasty in 1501, converted the Iranian people from Sunni Islam to Twelver Shia Islam, and made Twelver Shia Islam the official religion of Iran. Safavid conversion of Iran from Sunnism to Shiism marked one of the most important turning points in Iranian and Muslim history. Starting in 1736 under Nader Shah, Iran reached its greatest territorial extent since the Sassanid Empire, briefly possessing what was arguably the most powerful empire at the time.'), ('400', 'In 1387, Timur ordered the complete massacre of Isfahan, reportedly killing 70,000 citizens. The Ilkhans and the Timurids soon came to adopt the ways and customs of the Iranians, choosing to surround themselves with a culture that was distinctively Iranian. By the 1500s, Ismail I from Ardabil, established the Safavid Dynasty, with Tabriz as the capital.'), ('400', 'By the 1500s, Ismail I from Ardabil, established the Safavid Dynasty, with Tabriz as the capital. Beginning with Azerbaijan, he subsequently extended his authority over all of the Iranian territories, and established an intermittent Iranian hegemony over the vast relative regions, reasserting the Iranian identity within large parts of the Greater Iran. Iran was predominantly Sunni, but Ismail instigated a forced conversion to the Shia branch of Islam, by which the Shia Islam spread throughout the Safavid territories in the Caucasus, Iran, Anatolia, and Mesopotamia.'), ('400', 'The Ilkhans and the Timurids soon came to adopt the ways and customs of the Iranians, choosing to surround themselves with a culture that was distinctively Iranian. By the 1500s, Ismail I from Ardabil, established the Safavid Dynasty, with Tabriz as the capital. Beginning with Azerbaijan, he subsequently extended his authority over all of the Iranian territories, and established an intermittent Iranian hegemony over the vast relative regions, reasserting the Iranian identity within large parts of the Greater Iran.'), ('6', 'Meanwhile, the Chinese Ming dynasty fell to the rebellion of Li Zicheng (1606–1645) in 1644, yet his short-lived Shun dynasty was crushed by the Manchu invasion and the Han Chinese general Wu Sangui (1612–1678). China Daily states that when the following Qing dynasty replaced the Ming dynasty, it merely "strengthened administration of Tibet." However, Kolmaš states that the Dalai Lama was very observant of what was going on in China and accepted a Manchu invitation in 1640 to send envoys to their capital at Mukden in 1642, before the Ming collapsed.'), ('6', 'China Daily states that when the following Qing dynasty replaced the Ming dynasty, it merely "strengthened administration of Tibet." However, Kolmaš states that the Dalai Lama was very observant of what was going on in China and accepted a Manchu invitation in 1640 to send envoys to their capital at Mukden in 1642, before the Ming collapsed. Dawa Norbu, William Rockhill, and George N. Patterson write that when the Shunzhi Emperor (r. 1644–1661) of the subsequent Qing dynasty invited the fifth Dalai Lama Lozang Gyatso to Beijing in 1652, Shunzhi treated the Dalai Lama as an independent sovereign of Tibet.'), ('400', 'Iran became a major contributor to the Islamic Golden Age, producing many influential scientists, scholars, artists, and thinkers. Establishment of the Safavid Dynasty in 1501, converted the Iranian people from Sunni Islam to Twelver Shia Islam, and made Twelver Shia Islam the official religion of Iran. Safavid conversion of Iran from Sunnism to Shiism marked one of the most important turning points in Iranian and Muslim history.'), ('304', 'The Shunga Empire or Shunga Empire was an ancient Indian dynasty from Magadha that controlled vast areas of the Indian subcontinent from around 187 to 78 BCE. The dynasty was established by Pushyamitra Shunga, after the fall of the Maurya Empire. Its capital was Pataliputra, but later emperors such as Bhagabhadra also held court at Besnagar, modern Vidisha in Eastern Malwa.'), ('6', 'However, Kolmaš states that the Dalai Lama was very observant of what was going on in China and accepted a Manchu invitation in 1640 to send envoys to their capital at Mukden in 1642, before the Ming collapsed. Dawa Norbu, William Rockhill, and George N. Patterson write that when the Shunzhi Emperor (r. 1644–1661) of the subsequent Qing dynasty invited the fifth Dalai Lama Lozang Gyatso to Beijing in 1652, Shunzhi treated the Dalai Lama as an independent sovereign of Tibet. Patterson writes that this was an effort of Shunzhi to secure an alliance with Tibet that would ultimately lead to the establishment of Manchu rule over Mongolia.'), ('400', 'Safavid conversion of Iran from Sunnism to Shiism marked one of the most important turning points in Iranian and Muslim history. Starting in 1736 under Nader Shah, Iran reached its greatest territorial extent since the Sassanid Empire, briefly possessing what was arguably the most powerful empire at the time. During the 19th century, Iran irrevocably lost swaths of its territories in the Caucasus which made part of the concept of Iran for centuries, to neighboring Imperial Russia.')]
    new_list = filter_proper_noun(question_sentence, id_text_pairs)


#### Dependency Parsing

In [ ]:
def processDependencyParse(questionsentence):
    """

    Returns:
        head (list) - head of the sentence
        notHead (list) - not head element of the sentence
    """
    jar_path = 'nltk/stanford-corenlp-4.2.2/stanford-corenlp-4.2.2.jar'
    models_jar_path = 'nltk/stanford-corenlp-4.2.2-models-english.jar'
    parser = StanfordDependencyParser(path_to_jar = jar_path, path_to_models_jar = models_jar_path)
    head=[]
    notHead=[]
    result = parser.raw_parse(questionsentence)
    dependency = result.__next__()
    for dep in list(dependency.triples()):
        if (dep[0][0]!='What')and(dep[0][0]!='Who')and(dep[0][0]!='When'):
          head.append(dep[0][0])
    #print(list(set(head)))
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(questionsentence)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    #print(filtered_sentence)
    for word in filtered_sentence:
        if (word not in head) and (word!='When')and(word!='What')and(word!='Who')and(word!='?')and(word!="'s"):
            notHead.append(word)
    #print(notHead)
    return list(set(head)), notHead

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
if TEST:
    question_sentence = 'What was the capital of the Safavid Dynasty?'
    #question_sentence = 'What latitudes does Iran lie between?'

    head, notHead = processDependencyParse(question_sentence)

    print(head)
    print(notHead)

In [ ]:
def GetAllRelationalWord(word):
    AllRelationalWord=[]
    AllRelationalWord.append(word)
    AllRelationalWord.append(stem(word))
    AllRelationalWord.extend(Lemma([word]))
    AllRelationalWord.extend(Synonyms(word))
    AllRelationalWord.extend(Antonyms(word))
    AllRelationalWord.extend(Hyponyms(word))
    AllRelationalWord.extend(Hypernyms(word))
    AllRelationalWord.extend(Meronyms(word))
    AllRelationalWord.extend(Holonyms(word))
    return list(set(AllRelationalWord))

In [ ]:
#test
# AllRelationalWord=GetAllRelationalWord('name')
# print(AllRelationalWord)

In [ ]:
def getRelativeWord(head, notHead):
  headRelationalWord=[]
  notHeadRelationalWord=[]
  for headword in head:
    headRelationalWord.append(GetAllRelationalWord(headword))
  for notheadword in notHead:
    notHeadRelationalWord.append(GetAllRelationalWord(notheadword))
  #print(headRelationalWord)
  #print(notHeadRelationalWord)
  return headRelationalWord, notHeadRelationalWord

#getRelativeWord(head, notHead)

In [ ]:
#Initial score list
def createScoreList(new_list):
  lenOfCandidate=len(new_list)
  score=[]
  for i in range(0,lenOfCandidate):
    score.append(0)
  #print(score)
  return score, lenOfCandidate

#createScoreList(new_list)

In [ ]:
def initialscore(list,lenOfCandidate):
  score = [i for i in range(len(list))]
  for i in range(0,lenOfCandidate):
    score[i]=0
  #print(score)
  return list


In [ ]:
def candidateRelativeWord(new_list):
  sentencetokenlist=[]
  stemlist=[]
  Lemmalist=[]
  for i in range(0,len(new_list)):
    sentencetokenlist.append([])
    sentencetokenlist[i].extend(word_tokenize(new_list[i][1]))
  #print(sentencetokenlist)
  for i in range(0,len(new_list)):
    stemlist.append([])
    for word in sentencetokenlist[i]:
      #print(stem(word))
      stemlist[i].append(stem(word))
  #print(stemlist)
  for i in range(0,len(new_list)):
    Lemmalist.append([])
    for word in sentencetokenlist[i]:
      Lemmalist[i].extend(Lemma([word]))
  for i in range(0,len(new_list)):
    sentencetokenlist[i].extend(stemlist[i])
    sentencetokenlist[i].extend(Lemmalist[i])
  #print(sentencetokenlist)
  return sentencetokenlist

#candidateRelativeWord(new_list)



In [ ]:
def computeScore(score,lenOfCandidate,headRelationalWord,notHeadRelationalWord,sentencetokenlist):
  initialscore(score,lenOfCandidate)
  for i in range(0,lenOfCandidate):
    for j in range(0,len(headRelationalWord)):
      for word in headRelationalWord[j]:
        if word in sentencetokenlist[i]:
          score[i]=score[i]+2
          break
  #print(score)
  #initialscore(score,lenOfCandidate)
  for i in range(0,lenOfCandidate):
    for j in range(0,len(notHeadRelationalWord)):
      for word in notHeadRelationalWord[j]:
        if word in sentencetokenlist[i]:
          score[i]=score[i]+1
          break
  #print(score)
  #initialscore(score,lenOfCandidate)
  return score

#computeScore(score,lenOfCandidate,headRelationalWord,notHeadRelationalWord,sentencetokenlist)

In [ ]:
def getBestMatchIndex(score):
  #print(score)
  max_val = 0
  idx_max = 0
  for i in range(len(score)):
    if score[i] > max_val:
      max_val = score[i]
      idx_max = i
  #print(score, max_val, idx_max)
  return idx_max

#bestIndex=getBestMatchIndex(score)
#print(bestIndex)

In [ ]:
#find the best match sentence trigram
# BestMatchSentence=new_list[bestIndex]
# print(BestMatchSentence)

In [ ]:
# SplitBestMatch = []
# SplitBestMatch = sent_tokenize(BestMatchSentence[1])
# lenOfBestMatch= len(SplitBestMatch)
# print(SplitBestMatch)
# BestMatchsentencetokenlist=[]
# for i in range(0,len(SplitBestMatch)):
#   BestMatchsentencetokenlist.append([])
#   BestMatchsentencetokenlist[i].extend(word_tokenize(SplitBestMatch[i]))
# print(BestMatchsentencetokenlist)
# score2=[]
# for i in range(0,lenOfBestMatch):
#   score2.append(0)
# #print(score2)
# computeScore(score2,lenOfBestMatch,headRelationalWord,notHeadRelationalWord,BestMatchsentencetokenlist)
# bestIndexfinal=getBestMatchIndex(score2)
# #print(bestIndexfinal)
# Answer=[[],[]]
# Answer[0]=BestMatchSentence[0]
# Answer[1]=SplitBestMatch[bestIndexfinal]
# print(Answer)

In [ ]:
# #print best match answer
# print(Answer)

In [ ]:
#main
def getAnswer(new_list, question_sentence):
  head, notHead = processDependencyParse(question_sentence)
  headRelationalWord, notHeadRelationalWord = getRelativeWord(head, notHead)
  score,lenOfCandidate = createScoreList(new_list)
  sentencetokenlist = candidateRelativeWord(new_list)
  score = computeScore(score,lenOfCandidate,headRelationalWord,notHeadRelationalWord,sentencetokenlist)
  bestIndex = getBestMatchIndex(score)
  BestMatchSentence=new_list[bestIndex]
  SplitBestMatch = []
  SplitBestMatch = sent_tokenize(BestMatchSentence[1])
  lenOfBestMatch= len(SplitBestMatch)
  #print(SplitBestMatch)
  BestMatchsentencetokenlist=[]
  for i in range(0,len(SplitBestMatch)):
    BestMatchsentencetokenlist.append([])
    BestMatchsentencetokenlist[i].extend(word_tokenize(SplitBestMatch[i]))
  #print(BestMatchsentencetokenlist)
  score2=[]
  for i in range(0,lenOfBestMatch):
    score2.append(0)
  #print(score2)
  score2 = computeScore(score2,lenOfBestMatch,headRelationalWord,notHeadRelationalWord,BestMatchsentencetokenlist)
  bestIndexfinal=getBestMatchIndex(score2)
  #print(bestIndexfinal)
  Answer=[[],[]]
  Answer[0]=BestMatchSentence[0]
  Answer[1]=SplitBestMatch[bestIndexfinal]
  #print(Answer)
  return [tuple(Answer)]

#answer = getAnswer(new_list, question_sentence)

In [ ]:
#print answer
#print(answer)

#### Extract Question Type

In [ ]:
def extract_question_type(question):
    question = question.upper()
    word_list = word_tokenize(question)
    
    if 'WHO' in word_list:
        return 'WHO'
    if 'WHEN' in word_list:
        return 'WHEN'
    if 'WHAT' in word_list:
        return 'WHAT'

#### Check pos

In [ ]:
def extract_pos_set(sentence, pos_set):
    tag_tuple = postag(sentence)

    keyword=[]
    for i in range(len(tag_tuple)):
        if tag_tuple[i][1] in pos_set:
            keyword.append(tag_tuple[i][0])
    return keyword

In [ ]:
def check_pos(sentence, pos_set):
    '''
    Return True if there is pos_set in the sentence, return False otherwise
    '''
    pos = extract_pos_set(sentence, pos_set)

    if pos:
        return True
    else:
        return False

In [ ]:
def containing_pos(id_text_pairs, pos_set):
    '''
    filter if 
    '''
    new_list = []
    for id_text_pair in id_text_pairs:
        if check_pos(id_text_pair[1], pos_set):
            new_list.append(id_text_pair)
    return new_list

#### Pipeline


In [ ]:
def print_miss(document_number, ques_ans_pair, res):
    if document_number != '400':
        return 
    print(document_number + ' ' + str(len(res)))
    print('miss', end=' ')
    print(ques_ans_pair)
    for i in res:
        print(i)

In [ ]:
def print_correct(document_number, ques_ans_pair, res):
    if document_number != '400':
        return 
    print(document_number + ' ' + str(len(res)))
    print('hit', end=' ')
    print(ques_ans_pair)
    for i in res:
        print(i)

In [ ]:
def run_pipe(question):
    question_type = extract_question_type(question)
    res = extract_doc(question, 5)
    top_n = [i[0] for i in res]
    res = extract_n_grams(doc_range=top_n, question=question, n=10)

    # if question_type == 'WHO':
    #     res = containing_pos(id_text_pairs, {'NNP', 'NNPS'})
    # elif question_type == 'WHEN':
    #     res = containing_pos(id_text_pairs, {'CD'})

    res = getAnswer(res, question)
    return res

## Test Accuracy

In [ ]:
a = test_data['400']
smaller_testdata = {'400': a}

In [ ]:
if TEST:
    correct_count, total_count = evaluate_all(run_pipe, smaller_testdata, display_miss=True, display_correct=False)
    print(correct_count/total_count)
    print(correct_count)
    print(total_count)

## Run Test Question

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def run_test_question(input_path, output_path, filter_fn):
    questions = str()
    with open(input_path, 'r') as file:
        questions = file.read()
        file.close()
    questions = sent_tokenize(questions)

    answer_ids = list()
    answer_texts = list()
    for que in questions:
        ans = filter_fn(que)
        answer_ids.append(ans[0][0])
        answer_texts.append(ans[0][1])
    
    data = {'question':questions, 'support_id':answer_ids, 'support_sent':answer_texts}
    df = pd.DataFrame(data)

    df.to_csv(output_path)
    print(f"done, result in {output_path}")

In [ ]:
if TEST:
    run_test_question(test_input_path, test_output_path, run_pipe)

## Demo

In [ ]:
'''-------Task 1---------'''
filePath="../data/articles/articles/"
articleSentenceList, articleSentenceList2, articleTokenList, articleTokenList2 = Tokenize(filePath)
LemmaList = Lemmatize(articleTokenList2)
outputSentenceAndWord(articleSentenceList2,articleTokenList2,LemmaList)
#outputDependencyParse(articleSentenceList2)

In [ ]:
'''-------Task 2--------'''
input_path = './sample_questions.txt'
output_path = './test_questions.xlsx'
run_test_question(input_path, output_path, filter_fn=run_pipe)

done, result in ./test_questions.xlsx
